# Homework 3, BEE 6940 (Due By 4/13/23, 9:00PM)

**Name**: Demola Ogunnaike

**ID**: dko22 (5023990)

## Overview

### Instructions

- Problem 1 asks you to calibrate the Rahmstorf (2007) semi-empirical sea-level rise model using Markov chain Monte Carlo and `Turing.jl`.
- Problem 2 asks you to generate projections of global sea-level rise based on this model for different Representative Concentration Pathways.
- Problem 3 asks you to compare the results of this calibration with your bootstrap estimates from Lab 5.

### Load Environment

The following code loads the environment and makes sure all needed packages are installed. This should be at the start of most Julia scripts.

In [54]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `c:\Users\demol\Documents\BEE 6940\hw3-DemolaASC5`


In [55]:
using Random
using CSVFiles # load CSV data
using DataFrames # data storage and presentation
using Plots # plotting library
using StatsPlots # statistical plotting
using Distributions # statistical distribution interface
using Turing # probabilistic programming and MCMC
using Optim # optimization library


## Problems (100 points)

### Problem 1 (60 points)

In this problem, we will use Markov chain Monte Carlo to calibrate the [Rahmstorf (2007)](https://doi.org/10.1073/pnas.0907765106) semi-empirical sea-level rise model:
$$\frac{dH(t)}{dt} = \alpha (T(t) - T_0),$$
where $H(t)$ is global mean sea-level anomaly (in mm), $T(t)$ is the global mean surface temperature (in $^\circ$ C), $T_0$ is the temperature (in $^\circ$ C) when sea-level rise is in equilibrium, and $\alpha$ is the sea-level rise sensitivity to warming. Discretizing this equation with an annual time step yields
$$H(t+1) = H(t) + \alpha (T(t) - T_0).$$

Our goal in this problem is to calibrate this model and quantify parametric uncertainties using Markov chain Monte Carlo.

In [56]:
function rahmstorf_model(α, T₀, H₀, temp_data)
    temp_effect = α .* (temp_data .- T₀)
    slr_predict = cumsum(temp_effect) .+ H₀
    return slr_predict
end

rahmstorf_model (generic function with 1 method)

First, let's load the data.

In [57]:
# load data files
slr_data = DataFrame(load("data/CSIRO_Recons_gmsl_yr_2015.csv"))
gmt_data = DataFrame(load("data/NOAA_IPCC_RCPtempsscenarios.csv"))

slr_data[:, :Time] = slr_data[:, :Time] .- 0.5; # remove 0.5 from Times
dat = leftjoin(slr_data, gmt_data, on="Time") # join data frames on time

select!(dat, [1, 2, 6])  # drop columns we don't need
first(dat, 6)

┌ Warning: In data/NOAA_IPCC_RCPtempsscenarios.csv line 426 has 0 fields but 13 fields are expected. Skipping row.
└ @ TextParse C:\Users\demol\.julia\packages\TextParse\gNKVx\src\csv.jl:382


Row,Time,GMSL (mm),Historical NOAA temp & CNRM RCP 8.5 with respect to 20th century
,Float64,Float64,Float64?
1,1880.0,-158.7,-0.16
2,1881.0,-153.1,-0.1
3,1882.0,-169.9,-0.12
4,1883.0,-164.6,-0.17
5,1884.0,-143.7,-0.23
6,1885.0,-145.2,-0.2


In [58]:
slr_data = dat[:, 2];
temp_data = dat[:,3];

Next, let's specify the statistical model. Denote the residuals as $$z(\mathbf{x}, t)=y_t - F(\mathbf{x}, t),$$ where $y_t$ is the data at time $t$ and $F(\mathbf{x}, t)$ is the model output at time $t$ with parameters $\mathbf{x}$. 

For (relative) simplicity, we will assume $z(\mathbf{x}, t)$ is given as an AR(1) process, *i.e.* $$z(\mathbf{x}, t) = \rho z(\mathbf{x}, t-1) + \varepsilon_t, \quad \varepsilon_t \sim \text{Normal}(0, \sigma^2).$$

The unconditional variance of an AR(1) process is $$\text{Var}(z) = \frac{\sigma^2}{1-\rho^2},$$ so the likelihood for $z$ (abusing notation to only refer to the time-index, not the parameters) can be written as:
$$\begin{align*}
z_1 &\sim \text{Normal}\left(0, \frac{\sigma^2}{1-\rho^2}\right) \\
z_t &\sim \text{Normal}(\rho z_{t-1}, \sigma^2)
\end{align*}$$

#### Problem 1.1 (15 points)

Select priors for each of the parameters $\alpha$, $T_0$, $H_0$, $\rho$, and $\sigma^2$. Justify these priors based on relevant considerations of prior information and model structure and regularization/informativeness. 

The following priors were selected, based on the results from how priors were selected to fit data in previous labs. Priors were improved over time based on the results from the chain plot. 

α ~ Normal(0,2) <br>
T₀ ~ Normal(-1,1) <br>
H₀ ~ Normal(-200,100) <br> 

σ ~ Normal(7,2.5), with all numbers being greater than 0 <br>
ρ ~ Normal(0.5,0.01) <br>

#### Problem 1.2 (15 points)

Code the posterior model in `Turing.jl` syntax in the below function. As a tip, do not directly model the residuals (this will increase the model dimension due to how `Turing.jl` parses the model code), but write out the residuals in terms of the data and the model output and rearrange as discussed in class. In other words, use sampling commands like:

```julia
sl_predict = rahmstorf_model(...)
sl_data[1] ~ Normal(sl_predict[1], 2)
```
rather than 
```julia
residuals = rahmstorf_model(...) .- sl_data
residuals[1] ~ Normal(0, 2)
```

In [59]:
@model function slr_posterior(slr_data, temp_data)
    # define priors

    α ~ Normal(0,2)
    T₀ ~ Normal(-1,1) 
    H₀ ~ Normal(-200,100) 
    
    σ ~ Normal(7,2.5)
    ρ ~ Normal(0.5,0.01)

    σ_slr = sqrt(max((α^2)/(1-ρ^2),0))

    slr_predict = rahmstorf_model(rand(α), rand(T₀), rand(H₀), temp_data)
    slr_data[1] = Normal(slr_predict[1],σ_slr)

    # define likelihood
    for t = 1:length(sl_predict)
        slr_data[t] ~ Noraml(slr_predict[t] - p*(s1r_data[t-1]-slr_predict[t-1]),α)
    end 
end
slr_model = slr_posterior(slr_data, temp_data)

DynamicPPL.Model{typeof(slr_posterior), (:slr_data, :temp_data), (), (), Tuple{Vector{Float64}, Vector{Union{Missing, Float64}}}, Tuple{}, DynamicPPL.DefaultContext}(slr_posterior, (slr_data = [-158.7, -153.1, -169.9, -164.6, -143.7, -145.2, -147.6, -153.2, -151.1, -149.5  …  37.2, 37.3, 41.6, 43.5, 52.2, 58.0, 65.7, 67.3, 76.1, 67.7], temp_data = Union{Missing, Float64}[-0.16, -0.1, -0.12, -0.17, -0.23, -0.2, -0.18, -0.26, -0.17, -0.09  …  0.58, 0.65, 0.6, 0.59, 0.51, 0.6, 0.66, 0.53, 0.58, 0.62]), NamedTuple(), DynamicPPL.DefaultContext())

#### Problem 1.3 (15 points)

Draw samples from the posterior using MCMC. How did you decide how many chains to use and how many iterations to run the chain(s) for? How did you determine if your Markov chain(s) converged (include any plots and/or quantities which you used to make this assessment)?

Tip: If you find poor convergence or that your sampler is taking a while, look at your model definition. You may need to reconsider your priors, or you may be able to reformulate the model to make it more efficient. Or you might just need to increase the number of iterations, because a starting point took longer to converge.

In [67]:
chain = let
    model = slr_posterior(slr_data, temp_data) # load the data into the model object
    sampler = NUTS() # use the no-u-turn sampler (there are other options, but this is a good one)
    # replace these with your own values
    n_per_chain = 5000 # number of iterations per chain
    nchains = 4 # number of MCMC chains to run
    sample(model, sampler, MCMCThreads(), n_per_chain, nchains, drop_warmup=true) # uncomment for multiple threads, i.e. nchains > 1
    # sample(model, sampler, n_per_chain, drop_warmup=true) # comment out to use multiple chains
end

┌ Warning: Only a single thread available: MCMC chains are not sampled in parallel
└ @ AbstractMCMC C:\Users\demol\.julia\packages\AbstractMCMC\F9Hbk\src\sample.jl:296
Sampling (1 threads)   0%|█                             |  ETA: N/A
Sampling (1 threads) 100%|██████████████████████████████| Time: 0:00:00


CompositeException: TaskFailedException

    nested task error: TaskFailedException
    
        nested task error: ArgumentError: Sampler for this object is not defined
        Stacktrace:
          [1] Random.Sampler(#unused#::Type{TaskLocalRNG}, sp::Random.SamplerTrivial{Float64, Float64}, #unused#::Val{1})
            @ Random C:\Users\demol\AppData\Local\Programs\Julia-1.8.5\share\julia\stdlib\v1.8\Random\src\Random.jl:146
          [2] Random.Sampler(rng::TaskLocalRNG, x::Random.SamplerTrivial{Float64, Float64}, r::Val{1})
            @ Random C:\Users\demol\AppData\Local\Programs\Julia-1.8.5\share\julia\stdlib\v1.8\Random\src\Random.jl:140
          [3] rand(rng::TaskLocalRNG, X::Random.SamplerTrivial{Float64, Float64}) (repeats 2 times)
            @ Random C:\Users\demol\AppData\Local\Programs\Julia-1.8.5\share\julia\stdlib\v1.8\Random\src\Random.jl:254
          [4] rand(X::Float64)
            @ Random C:\Users\demol\AppData\Local\Programs\Julia-1.8.5\share\julia\stdlib\v1.8

#### Problem 1.4 (15 points)

Make a corner/pairs plot of the posterior samples using `corner(chain[:, :, 1])` (unfortunately, this plot recipe does not work well with multiple chains, but if your chains have converged and were run long enough, they should all look roughly the same). Do the correlations between the various parameters make sense to you? Why or why not?

In [62]:
plot(chain)

UndefVarError: UndefVarError: chain not defined

### Problem 2 (30 points)

In this problem, we will use our calibrated model above with NOAA's RCP8.5 temperature projections to look at different point estimates of the model parameters and now their associated projections compare to the posterior predictive probability distribution.

#### Problem 2.1 (5 points)

Compute the maximum *a posteriori* (MAP) and maximum likelhood (MLE) estimates for the model.

#### Problem 2.2 (15 points)

Use `gmt_data` (the first column for the years and the fourth column for temperatures) to simulate sea-levels using your model for 1,000 posterior parameter samples, as well as for the MLE and MAP estimates. Plot the 95% quantiles by year for the posterior predictive samples along with the posterior median and the MAP and MLE projections.

#### Problem 2.3 (10 points)

Based on your plot in Problem 2.2, discuss the potential implications for risk assessment of using the different point estimates (MAP and MLE) vs the broader posterior predictive ensemble. When would you suggest that these estimates be used or not used?